In [1]:
import json
import numpy as np


def calculate_polys(data: list[dict], x_label: str, y_label: str):
    freqs: list[str] = {p['freq'] for p in data}
    polys: dict[dict] = {}

    for freq in freqs:
        points = [p for p in data if p['freq'] == freq]

        x = [0, *[p[x_label] for p in points]]
        y = [0, *[p[y_label] for p in points]]
        # x = [p[x_label] for p in points]
        # y = [p[y_label] for p in points]

        power = 2
        temp = np.poly1d(np.polyfit(x, y, power))

        alpha = 'abcdefghijklmnopqrstuvwxyz'
        poly = {}
        for i, p in enumerate(range(power, -1, -1)):
            poly[alpha[i]] = round(temp[p], 10)

        polys[freq] = poly

    return polys


def create_poly_header(polys):
    header = []

    # make sure the outputs are in order
    bands = [p for p in polys['fwd']]
    bands.sort()

    header.append('polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['fwd'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    header.append('polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['rev'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    return ''.join(header)


with open('fwd.json', 'r') as f:
    fwd = json.load(f)['data']

with open('rev.json', 'r') as f:
    rev = json.load(f)['data']

polys = {
    'fwd': calculate_polys(fwd, 't_fwd_volts', 'm_fwd'), 
    'rev': calculate_polys(rev, 't_rev_volts', 'm_fwd')
}


header = create_poly_header(polys)

print(header)

polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {
    {1.51587e-05, -0.0060328824, 2.7082489741}, // 01800000
    {1.20743e-05, -0.0056207022, 2.7441929053}, // 03500000
    {1.11827e-05, -0.0044110833, 2.3148621334}, // 07000000
    {1.1779e-05, -0.0049848463, 2.5213582082}, // 10100000
    {1.18646e-05, -0.0049070542, 2.6061124915}, // 14000000
    {1.13141e-05, -0.00418468, 2.2563557919}, // 18068000
    {1.15549e-05, -0.0056224365, 2.9218492332}, // 21000000
    {1.13151e-05, -0.0042982617, 2.2547151272}, // 24890000
    {1.16743e-05, -0.004352165, 2.3111931975}, // 28000000
    {1.04125e-05, 0.0041391385, -0.4051507692}, // 50000000
};

polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {
    {8.7673e-06, 0.0013695546, -0.0372789455}, // 01800000
    {6.6581e-06, 0.0011797294, -0.0075654381}, // 03500000
    {6.1815e-06, 0.0013303914, -0.0230372892}, // 07000000
    {6.3509e-06, 0.0014253321, -0.0325788626}, // 10100000
    {6.3027e-06, 0.0014752539, -0.0303042619}, // 1400